In [ ]:
from unsloth import FastLanguageModel
from ollama import chat
from datasets import Dataset

In [ ]:
import pandas as pd

import src.inference as inference
from src.utils import run_training

In [ ]:
# Run Training, Test and saving 
model, tokenizer = run_training('aduc') # task names among [aduc, claim_detection, evidence_detection, evidence_type, fallacies, relation, stance_detection, mt_ft]

In [ ]:
# Test one model on the test sample of each tasks

all_result = inference.inference_on_all_data(
    model_for_task='corrected',
    model_to_use='few-shot',
    few_shot=True,
)
all_result

In [ ]:
# DeBERTa

from deberta_fn import get_datasets, compute_metrics, tokenize_fn

task_name = 'aduc' # task names among [aduc, claim_detection, evidence_detection, evidence_type, fallacies, relation, stance_detection, mt_ft]
output_dir = f'./outputs/{task_name}/deberta_{task_name}'
model_name = 'microsoft/deberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)

train_dataset, test_dataset = get_datasets(tokenizer, tokenize_fn, task_name)

# raise ValueError
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="no",
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    learning_rate=3e-5,
    optim="adamw_torch_fused",
    weight_decay=0.01,
    logging_steps=1,
    report_to="tensorboard",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # eval_on_start=True,
    remove_unused_columns=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Test the model
test_results = trainer.evaluate(test_dataset)
print(test_results)
test_results
test_df = pd.Series(data=test_results)
test_df.to_csv(f'./test_res/{task_name}/deberta_res_{task_name}.csv', header=['score'])